# Spark Apache (семинары)
## Урок 2. Инструменты работы и визуализации ч.1

**Условие:** дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

Данные таблицы:

| Id   | name  | Salary | managerId |
|------|-------|--------|-----------|
| 1    | Joe   | 70     | 3         |
| 2    | Henry | 80     | 4         |
| 3    | Sam   | 60     | Null      |
| 4    | Max   | 90     | null      |

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

In [2]:
# Инициализация Spark сессии
spark = SparkSession.builder \
    .appName("EmployeeSalaryComparisonFromCSV") \
    .getOrCreate()

In [3]:
# Схема данных
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("managerid", IntegerType(), True) 
])

In [4]:
# Исходные данные для таблицы
data = [
    (1, "Joe", 70, 3),
    (2, "Henry", 80, 4),
    (3, "Sam", 60, None),
    (4, "Max", 90, None)
]

In [5]:
# Вариант 1. Создаем DataFrame из списка кортежей 
employees_df = spark.createDataFrame(data, schema=schema)

print("Список сотрудников:")
employees_df.show()

Список сотрудников:
+---+-----+------+---------+
| id| name|salary|managerid|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [ ]:
# Вариант 2a. Читаем данные из CSV файла по схеме
employees_df = spark.read \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("nullValue", "") \
    .schema(schema) \
    .csv("employees.csv")

print("Список сотрудников:")
employees_df.show()

Список сотрудников:
+---+-----+------+---------+
| id| name|salary|managerid|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [ ]:
# Вариант 2b. Читаем данные из CSV файла с автоматическим определением схемы
employees_df = spark.read.load('employees.csv', format="csv", sep=";", header="true", Infer_schema=True)

print("Список сотрудников:")
employees_df.show()

Список сотрудников:
+---+-----+------+---------+
| id| name|salary|managerid|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [ ]:
# Сотрудники, которые зарабатывают больше своих менеджеров
detailed_result = employees_df.alias("emp") \
    .join(employees_df.alias("mgr"), 
          col("emp.managerId") == col("mgr.Id"), 
          "inner") \
    .filter(col("emp.Salary") > col("mgr.Salary")) \
    .select(
        col("emp.name").alias("employee"),
        col("emp.Salary").alias("employee_salary"),
        col("mgr.name").alias("manager"),
        col("mgr.Salary").alias("manager_salary")
    )

print("Сотрудники, которые зарабатывают больше своих менеджеров:")
result.show()


Подробный результат:
+--------+---------------+-------+--------------+
|employee|employee_salary|manager|manager_salary|
+--------+---------------+-------+--------------+
|     Joe|             70|    Sam|            60|
+--------+---------------+-------+--------------+



In [4]:
spark.stop()